In [3]:
from func_def import *
events = get_links(url='http://www.ufcstats.com/statistics/events/completed?page=all')
events.date = pd.to_datetime(events.date)
events = events[events.date < get_current_date()].reset_index(drop=True)
events.to_csv('event_list.csv')
events

,date,event_url
0,2023-11-04,http://www.ufcstats.com/event-details/7c4ec656...
1,2023-10-21,http://www.ufcstats.com/event-details/13a0fb8f...
2,2023-10-14,http://www.ufcstats.com/event-details/f3a07827...
3,2023-10-07,http://www.ufcstats.com/event-details/c8a49ff2...
4,2023-09-23,http://www.ufcstats.com/event-details/c945adc2...
...,...,...
663,1995-07-14,http://www.ufcstats.com/event-details/1c3f5e85...
664,1995-04-07,http://www.ufcstats.com/event-details/dedc3bb4...
665,1994-12-16,http://www.ufcstats.com/event-details/b60391da...
666,1994-09-09,http://www.ufcstats.com/event-details/1a49e067...


In [4]:
#get existing data
UPDATE_ALL = True

exist_data = []
exist_events_url = []

if not UPDATE_ALL:
    try:
        exist_data = pd.read_csv('data_pre_processed.csv',index_col=0)
        exist_events_url = list(np.unique(exist_data.event_url))
    except:
        print('Cant find data_pre_processed.csv')
        exist_data = []
        exist_events_url = []

print(f'Current Existing events: {len(exist_events_url)}')

Current Existing events: 0


In [5]:
#get new data
new_events = events[~events.event_url.isin(exist_events_url)].reset_index(drop=True)
#new_events = events
print(f'New events: {len(new_events)}')
new_events.head(3)

New events: 668


,date,event_url
0,2023-11-04,http://www.ufcstats.com/event-details/7c4ec656...
1,2023-10-21,http://www.ufcstats.com/event-details/13a0fb8f...
2,2023-10-14,http://www.ufcstats.com/event-details/f3a07827...


In [6]:
new_events.event_url.tolist()

['http://www.ufcstats.com/event-details/7c4ec656d8fcb867',
 'http://www.ufcstats.com/event-details/13a0fb8fbdafb54f',
 'http://www.ufcstats.com/event-details/f3a078277b3b8ff4',
 'http://www.ufcstats.com/event-details/c8a49ff2acb6f3c5',
 'http://www.ufcstats.com/event-details/c945adc22c2bfe8f',
 'http://www.ufcstats.com/event-details/8fa2b06572365321',
 'http://www.ufcstats.com/event-details/ece280745f8727b8',
 'http://www.ufcstats.com/event-details/ef61d9f5176b3200',
 'http://www.ufcstats.com/event-details/89a407032911e27e',
 'http://www.ufcstats.com/event-details/2719f300b0439039',
 'http://www.ufcstats.com/event-details/d2fa318f34d0aadc',
 'http://www.ufcstats.com/event-details/6f81b6de2557739a',
 'http://www.ufcstats.com/event-details/ccd58ff71e260ed5',
 'http://www.ufcstats.com/event-details/1174782eacde9b0c',
 'http://www.ufcstats.com/event-details/c9885b1b7c7055a0',
 'http://www.ufcstats.com/event-details/6085ceb59087514b',
 'http://www.ufcstats.com/event-details/e9e1acc96536bb4f

In [7]:
new_data = []
for index in tqdm(range(len(new_events))):
    try:
        new_data.extend(get_event_data(EVENT_URL=new_events.loc[index,'event_url'],EVENT_TIME=new_events.loc[index,'date']))
    except:
        continue   

try:
    data = pd.concat([exist_data,pd.DataFrame(new_data)]).reset_index(drop=True)
except:
    data = new_data

data[-3:-1]

100%|██████████| 668/668 [4:41:35<00:00, 25.29s/it]  


[{'url': 'http://www.ufcstats.com/fight-details/ccee020be2e8d9f6',
  'event_url': 'http://www.ufcstats.com/event-details/a6a9ab5a824e8f66',
  'date': Timestamp('1994-03-11 00:00:00'),
  'result': 'W',
  'fighter': 'Johnny Rhode',
  'opponent': 'David Levick',
  'title': 'Open Weight Bout',
  'method': 'KO/TKO',
  'round': '1',
  'time': '12:13',
  'format': 'No Time Limit',
  'f_fighter': 'Johnny Rhodes',
  'f_kd': '0',
  'f_sig_str': '11 of 17',
  'f_sig_str%': '64%',
  'f_total_str': '74 of 86',
  'f_td': '1 of 1',
  'f_td%': '100%',
  'f_sub_att': '0',
  'f_rev': '0',
  'f_ctrl': '--',
  'o_fighter': 'David Levicki',
  'o_kd': '0',
  'o_sig_str': '4 of 5',
  'o_sig_str%': '80%',
  'o_total_str': '95 of 102',
  'o_td': '0 of 0',
  'o_td%': '---',
  'o_sub_att': '0',
  'o_rev': '0',
  'o_ctrl': '--',
  'f_head_str': '9of15',
  'f_body_str': '1of1',
  'f_leg_str': '1of1',
  'f_distance_str': '1of1',
  'f_clinch_str': '1of1',
  'f_ground_str': '9of15',
  'o_head_str': '4of5',
  'o_body_

In [9]:
pd.DataFrame(data).to_csv('data_pre_processed.csv',index=[0])

In [7]:
# Test unit
FIGHT_URL = 'http://www.ufcstats.com/fight-details/ac37849f627a79c4'
get_fight_general_stats(FIGHT_URL=FIGHT_URL)

{'f_fighter': 'Jailton Almeida',
 'f_kd': '0',
 'f_sig_str': '38 of 56',
 'f_sig_str%': '67%',
 'f_total_str': '120 of 153',
 'f_td': '6 of 15',
 'f_td%': '40%',
 'f_sub_att': '4',
 'f_rev': '1',
 'f_ctrl': '21:10',
 'o_fighter': 'Derrick Lewis',
 'o_kd': '0',
 'o_sig_str': '20 of 39',
 'o_sig_str%': '51%',
 'o_total_str': '28 of 54',
 'o_td': '0 of 2',
 'o_td%': '0%',
 'o_sub_att': '0',
 'o_rev': '0',
 'o_ctrl': '0:57'}

In [2]:
get_event_data(EVENT_URL='http://www.ufcstats.com/event-details/7c4ec656d8fcb867')

[{'url': 'http://www.ufcstats.com/fight-details/ac37849f627a79c4',
  'event_url': 'http://www.ufcstats.com/event-details/7c4ec656d8fcb867',
  'date': None,
  'result': 'W',
  'fighter': 'Jailton Almeida',
  'opponent': 'Derrick Lewis',
  'title': 'Heavyweight Bout',
  'method': 'Decision - Unanimous',
  'round': '5',
  'time': '5:00',
  'format': '5 Rnd (5-5-5-5-5)',
  'f_fighter': 'Jailton Almeida',
  'f_kd': '0',
  'f_sig_str': '38 of 56',
  'f_sig_str%': '67%',
  'f_total_str': '120 of 153',
  'f_td': '6 of 15',
  'f_td%': '40%',
  'f_sub_att': '4',
  'f_rev': '1',
  'f_ctrl': '21:10',
  'o_fighter': 'Derrick Lewis',
  'o_kd': '0',
  'o_sig_str': '20 of 39',
  'o_sig_str%': '51%',
  'o_total_str': '28 of 54',
  'o_td': '0 of 2',
  'o_td%': '0%',
  'o_sub_att': '0',
  'o_rev': '0',
  'o_ctrl': '0:57',
  'f_head_str': '32of48',
  'f_body_str': '5of5',
  'f_leg_str': '1of3',
  'f_distance_str': '5of7',
  'f_clinch_str': '3of8',
  'f_ground_str': '30of41',
  'o_head_str': '17of33',
  'o